In [20]:
from pydantic import BaseModel
from pydantic import ValidationError


class AppConfig(BaseModel):
    debug: bool = False
    max_users: int = 100
    db_url: str

config_data = {
    "debug": True,
    "db_url": "postgresql://localhost/mydb",
}

app_config = AppConfig(**config_data)

# Validate the config data
print(app_config.debug)
print(app_config.max_users)
print(app_config.db_url)

# should convert to False
app_config = AppConfig(debug='False', db_url="postgresql://localhost/mydb",)
assert not app_config.debug

# should fail because we cannot convert 'asdf' to bool
import pytest
with pytest.raises(ValidationError) as exc_info:
    app_config = AppConfig(debug='asdf', db_url="postgresql://localhost/mydb")

True
100
postgresql://localhost/mydb


In [22]:
from dataclasses import dataclass

@dataclass
class AppConfig():
    db_url: str
    debug: bool = False
    max_users: int = 100

# this works, whereas pydantic fails because it's an unexpected value
app_config = AppConfig(debug='asdf', db_url="postgresql://localhost/mydb")
